In [15]:
import pandas as pd
import json
from typing import Dict, List
import openai
from dotenv import load_dotenv
import os
import sys
sys.path.append('d:/perso/personal/Desktop Files/LLM projects/Elevate Repo/PowerBI-LLM')
from functions.upload_powerbi_files import *
from functions.extract_original_json import *

In [2]:
# Load environment variables
load_dotenv()
# Get API key from environment variable
openai.api_key = os.getenv("OPENAI_API_KEY")

In [12]:
zip_file = 'example reports/contenus_ftv.zip'
report_json_content, model_bim_content, inner_folder_path, report_json_path, model_bim_path = extract_report_and_model(zip_file)

In [13]:
report_json_content

{'config': '{"version":"5.61","themeCollection":{"baseTheme":{"name":"CY22SU03","version":"5.32","type":2},"customTheme":{"name":"Data_Provider6027261247871858.json","version":"5.52","type":1}},"activeSectionIndex":2,"bookmarks":[{"displayName":"Sous categ","name":"Bookmarkbc9681d14f89cc9048cb","explorationState":{"version":"1.3","activeSection":"ReportSection6cbc176f03d156d7ceb7","sections":{"ReportSection6cbc176f03d156d7ceb7":{"filters":{"byExpr":[{"name":"Filterb4dcd77b087ae66633e9","type":"Categorical","filter":{"Version":2,"From":[{"Name":"c","Entity":"contenu_recrutant_Refonte","Type":0}],"Where":[{"Condition":{"Not":{"Expression":{"In":{"Expressions":[{"Column":{"Expression":{"SourceRef":{"Source":"c"}},"Property":"replay_or_direct"}}],"Values":[[{"Literal":{"Value":"null"}}]]}}}}}]},"expression":{"Column":{"Expression":{"SourceRef":{"Entity":"contenu_recrutant_Refonte"}},"Property":"replay_or_direct"}},"howCreated":1}]},"visualContainers":{"2edb91c281c07723b226":{"filters":{"by

In [ ]:
def extract_relevant_elements_slicer_unif(json_data):
    extracted_data = {
        #"config": json_data.get("config", {}),
        "sections": []
    }
    for section in json_data.get("sections", []):
        # Store displayName to know the section/page name
        section_summary = {
            "displayName": section.get("displayName", ""),
            "visualContainers": []
        }
        # Process each visual container in the section
        for visual in section.get("visualContainers", []):
            # Parse config if it's a string
            config_data = visual.get("config", {})
            if isinstance(config_data, str):
                config_data = json.loads(config_data)
            # Extract relevant visual properties
            visual_type = config_data.get("singleVisual", {}).get("visualType", "")
            if visual_type in ["slicer", "advancedSlicerVisual"]:
                visual_summary = {
                    "name": config_data.get("name", ""),
                    "visualType": visual_type,
                    "prototypeQuery": config_data.get("singleVisual", {}).get("prototypeQuery", {}),
                    "objects": config_data.get("singleVisual", {}).get("objects", {}),
                    "vcObjects": config_data.get("singleVisual", {}).get("vcObjects", {})
                }
                # Add visual summary if it has useful data
                section_summary["visualContainers"].append(visual_summary)
        # Add section summary if it has relevant visual containers
        if section_summary["visualContainers"]:
            extracted_data["sections"].append(section_summary)
    return extracted_data

In [16]:
visuals = extract_relevant_elements_slicer_unif(report_json_content)
visuals

{'sections': [{'displayName': 'Récapitulatif dernière semaine',
   'visualContainers': [{'name': '1b4769cf076d0d584063',
     'visualType': 'slicer',
     'prototypeQuery': {'Version': 2,
      'From': [{'Name': 'r', 'Entity': 'Reference - MyCalendar', 'Type': 0}],
      'Select': [{'HierarchyLevel': {'Expression': {'Hierarchy': {'Expression': {'PropertyVariationSource': {'Expression': {'SourceRef': {'Source': 'r'}},
             'Name': 'Variation',
             'Property': 'Date'}},
           'Hierarchy': 'Hiérarchie de dates'}},
         'Level': 'Année'},
        'Name': 'Reference - MyCalendar.Date.Variation.Hiérarchie de dates.Année',
        'NativeReferenceName': 'Date Année'}],
      'OrderBy': [{'Direction': 1,
        'Expression': {'HierarchyLevel': {'Expression': {'Hierarchy': {'Expression': {'PropertyVariationSource': {'Expression': {'SourceRef': {'Source': 'r'}},
              'Name': 'Variation',
              'Property': 'Date'}},
            'Hierarchy': 'Hiérarchie 